In [1]:
import pandas as pd
import string
import numpy as np

In [2]:
#%%
import os
print(os.getcwd())

d:\OneDrive - MNSCU\Research\2023_AI_Text_Classification\AI_Generated_Text_Classfier


In [3]:
df1 = pd.read_csv('./data/wiki_features/wiki_features_GT.csv')
df1['Label'] = 0 #ground truth - human text
print(df1.shape)

(3974, 50786)


In [4]:
df2 = pd.read_csv('./data/wiki_features/wiki_features_ChatGPT.csv')
df2['Label'] = 1 #ChatGPT response
print(df2.shape)

(4557, 50786)


In [5]:
merged_df = pd.concat([df1, df2], ignore_index = True, sort = False)
merged_df.dropna(inplace=True)
print(merged_df.shape)
print(merged_df.columns)

(8530, 50786)
Index(['Unnamed: 0', 'indexes', 'word_count', 'char_count', 'word_density',
       'punctuation_count', 'title_word_count', 'upper_case_word_count',
       'noun_count', 'verb_count',
       ...
       '4991.2', '4992.2', '4993.2', '4994.2', '4995.2', '4996.2', '4997.2',
       '4998.2', '4999.2', 'Label'],
      dtype='object', length=50786)


In [8]:
X = merged_df.iloc[:, 2:-1]
y = merged_df['Label']
print(X.shape, y.shape)

(8530, 50783) (8530,)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = pd.DataFrame(X_train)
X_train = X_train.reset_index(drop=True)
X_test = pd.DataFrame(X_test)
X_test = X_test.reset_index(drop=True)
y_train = pd.DataFrame(y_train)
y_train = y_train.reset_index(drop=True)
y_test = pd.DataFrame(y_test)
y_test = y_test.reset_index(drop=True)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(5715, 50783) (5715, 1) (2815, 50783) (2815, 1)


In [10]:
import pickle
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
#train random forest
rf_clf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, \
                                min_samples_split=10, min_samples_leaf=10, min_weight_fraction_leaf=0.0, \
                                max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, \
                                bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, \
                                warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

print("Random Forest Model fitting started ....")
rf_model = rf_clf.fit(X_train, y_train)
pickle.dump(rf_model, open('./model/wiki_rf.pkl', 'wb'))
print('RF Model saved')
y_pred = rf_clf.predict(X_test)
y_pred = np.where(y_pred <= 0.5, 0, 1)
f1 = f1_score(y_test, y_pred)
print('RF - F1 Score = ', f1)
print(classification_report(y_test, y_pred, digits=4))

Random Forest Model fitting started ....


c:\Users\nttru\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == "":


RF Model saved
Rf - F1 Score =  0.9993311036789299
              precision    recall  f1-score   support

           0     0.9985    1.0000    0.9992      1319
           1     1.0000    0.9987    0.9993      1496

    accuracy                         0.9993      2815
   macro avg     0.9992    0.9993    0.9993      2815
weighted avg     0.9993    0.9993    0.9993      2815



In [11]:
#train SVM model
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

svm_clf = svm.SVC(C=4.0, kernel='rbf', degree=7, gamma='scale', coef0=0.0, shrinking=True, \
                    probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, \
                    max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

print("SVM Model fitting started ....")
svm_model = svm_clf.fit(X_train, y_train)
pickle.dump(svm_model, open('./model/wiki_svm.pkl', 'wb'))
print('SVM Model saved')
y_pred = svm_clf.predict(X_test)
y_pred = np.where(y_pred <= 0.5, 0, 1)
f1 = f1_score(y_test, y_pred)
print('SVM - F1 Score = ', f1)
print(classification_report(y_test, y_pred, digits=4))

SVM Model fitting started ....


c:\Users\nttru\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVM Model saved
SVM - F1 Score =  0.7650485436893204
              precision    recall  f1-score   support

           0     0.7428    0.6876    0.7142      1319
           1     0.7415    0.7901    0.7650      1496

    accuracy                         0.7421      2815
   macro avg     0.7422    0.7389    0.7396      2815
weighted avg     0.7421    0.7421    0.7412      2815



In [12]:
#XGB model
#import shap
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

xgb_clf = GradientBoostingClassifier(learning_rate=0.01, n_estimators=1000)
print("XGB Model fitting started ....")
model = xgb_clf.fit(X_train, y_train)
pickle.dump(model, open('./model/wiki_xgb.pkl', 'wb'))
print('Model saved')
y_pred = xgb_clf.predict(X_test)
y_pred = np.where(y_pred <= 0.5, 0, 1)
f1 = f1_score(y_test, y_pred)
print('F1 Score = ', f1)
print(classification_report(y_test, y_pred, digits=4))
'''
explainer = shap.Explainer(model)
X_train = pd.DataFrame(X_train)
shap_values = explainer(X_train)
# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])
'''

XGB Model fitting started ....


c:\Users\nttru\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Model saved
F1 Score =  0.9993315508021391
              precision    recall  f1-score   support

           0     0.9992    0.9992    0.9992      1319
           1     0.9993    0.9993    0.9993      1496

    accuracy                         0.9993      2815
   macro avg     0.9993    0.9993    0.9993      2815
weighted avg     0.9993    0.9993    0.9993      2815



"\nexplainer = shap.Explainer(model)\nX_train = pd.DataFrame(X_train)\nshap_values = explainer(X_train)\n# visualize the first prediction's explanation\nshap.plots.waterfall(shap_values[0])\n"